# Unified Agent Notebook (Playground → Code)

This notebook contains **everything in one place**:
1) install deps, 
2) define & save `agent.py`, 
3) quick test, 
4) MLflow log (runs:/), 
5) register in Unity Catalog, 
6) optional deploy.

**What’s fixed vs the auto-generated one:**
- Uses an **explicit UC Function client** to avoid default-auth errors
- Logs the model as a **run artifact** (so we always get `runs:/...`)
- Registers to UC using **`MlflowClient.create_model_version`** with the `runs:/...` source
- No wildcard tool names (only explicit UC functions) to prevent resolution issues


In [0]:
# Install dependencies
%pip install -U databricks-openai databricks-agents "mlflow-skinny[databricks]" databricks-sdk unitycatalog-ai backoff uv
try:
    dbutils.library.restartPython()
except Exception as e:
    print("Note: restartPython is available in Databricks notebooks only. Continuing...", e)

In [0]:
# Set tracking/registry URIs and UC context **before** any logging/registration
import mlflow

# URIs
mlflow.set_tracking_uri("databricks")
mlflow.set_registry_uri("databricks-uc")

# Target UC location (EDIT HERE if needed)
catalog = "wwc2025"
schema = "period_pipeline"
model_name = "wellness_agent"

UC_MODEL_NAME = f"{catalog}.{schema}.{model_name}"

# Make sure session is in the right catalog/schema (Databricks runtime)
try:
    spark.sql(f"USE CATALOG {catalog}")
    spark.sql(f"USE SCHEMA {schema}")
except Exception as e:
    print("Skipping USE CATALOG/SCHEMA (likely not on Databricks runtime):", e)

print("Tracking URI:", mlflow.get_tracking_uri())
print("Registry URI:", mlflow.get_registry_uri())
print("UC target:", UC_MODEL_NAME)

In [0]:
%%writefile agent.py
# agent.py — tool-calling agent with explicit UC function client

import json
from typing import Any, Callable, Generator, Optional
from uuid import uuid4
import warnings

import mlflow
from databricks.sdk import WorkspaceClient
from databricks_openai import UCFunctionToolkit
from mlflow.entities import SpanType
from mlflow.pyfunc import ResponsesAgent
from mlflow.types.responses import (
    ResponsesAgentRequest,
    ResponsesAgentResponse,
    ResponsesAgentStreamEvent,
)
from openai import OpenAI
from pydantic import BaseModel
from unitycatalog.ai.core.databricks import DatabricksFunctionClient


# ---- LLM endpoint (EDIT if you use a different one) ----
LLM_ENDPOINT_NAME = "databricks-meta-llama-3-3-70b-instruct"

# ---- System prompt (kept simple to avoid formatting pitfalls) ----
SYSTEM_PROMPT = (
    """You are a Women's Health and Wellness Advisor Agent built on Databricks.
You analyze period and symptom data to provide educational and supportive insights.
- You are not a doctor; always include a disclaimer.
- Use empathetic, supportive tone and simple explanations.
- When explaining results, focus on patterns and what they might suggest, not diagnoses.
- Use safe, evidence-based general information.
- Never share or infer private data across users. 
If the user does not provide a user_id, never call tools that require p_user_id. Prefer aggregate SQL over governed gold tables or call aggregate-capable tools only."""
)

class ToolInfo(BaseModel):
    name: str
    spec: dict
    exec_fn: Callable

def create_tool_info(tool_spec: dict, exec_fn_param: Optional[Callable] = None) -> "ToolInfo":
    tool_spec.get("function", {}).pop("strict", None)
    tool_name = tool_spec["function"]["name"]
    udf_name = tool_name.replace("__", ".")

    def exec_fn(**kwargs):
        function_result = UC_FUNCTION_CLIENT.execute_function(udf_name, kwargs)
        if function_result.error is not None:
            return function_result.error
        else:
            return function_result.value

    return ToolInfo(name=tool_name, spec=tool_spec, exec_fn=exec_fn_param or exec_fn)

# ---- Explicit UC Function client (avoid default auth discovery issues) ----
WORKSPACE_CLIENT = WorkspaceClient()
UC_FUNCTION_CLIENT = DatabricksFunctionClient(workspace_client=WORKSPACE_CLIENT)

# ---- List **explicit** UC function names (no wildcards) ----
UC_TOOL_NAMES = [
    "wwc2025.period_pipeline.recommend_wellness_bundle",
    "wwc2025.period_pipeline.get_short_intervals",
    "wwc2025.period_pipeline.find_users_by_avg_cycle_length",
    "wwc2025.period_pipeline.get_abnormal_cycles",
    "wwc2025.period_pipeline.get_user_metrics",
    "wwc2025.period_pipeline.get_long_bleeds",
    "wwc2025.period_pipeline.predict_next_period_row",
]

# Build tools with the explicit client
uc_toolkit = UCFunctionToolkit(function_names=UC_TOOL_NAMES, client=UC_FUNCTION_CLIENT)
TOOL_INFOS = [create_tool_info(tool_spec) for tool_spec in uc_toolkit.tools]

class ToolCallingAgent(ResponsesAgent):
    def __init__(self, llm_endpoint: str, tools: list[ToolInfo]):
        self.llm_endpoint = llm_endpoint
        self.workspace_client = WORKSPACE_CLIENT
        self.model_serving_client: OpenAI = self.workspace_client.serving_endpoints.get_open_ai_client()
        self._tools_dict = {tool.name: tool for tool in tools}

    def get_tool_specs(self) -> list[dict]:
        return [tool_info.spec for tool_info in self._tools_dict.values()]

    @mlflow.trace(span_type=SpanType.TOOL)
    def execute_tool(self, tool_name: str, args: dict) -> Any:
        return self._tools_dict[tool_name].exec_fn(**args)

    def call_llm(self, messages: list[dict[str, Any]]) -> Generator[dict[str, Any], None, None]:
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", message="PydanticSerializationUnexpectedValue")
            for chunk in self.model_serving_client.chat.completions.create(
                model=self.llm_endpoint,
                messages=self.prep_msgs_for_cc_llm(messages),
                tools=self.get_tool_specs(),
                stream=True,
            ):
                yield chunk.to_dict()

    def handle_tool_call(self, tool_call: dict[str, Any], messages: list[dict[str, Any]]) -> ResponsesAgentStreamEvent:
        args = json.loads(tool_call["arguments"])
        result = str(self.execute_tool(tool_name=tool_call["name"], args=args))
        tool_call_output = self.create_function_call_output_item(tool_call["call_id"], result)
        messages.append(tool_call_output)
        return ResponsesAgentStreamEvent(type="response.output_item.done", item=tool_call_output)

    def call_and_run_tools(self, messages: list[dict[str, Any]], max_iter: int = 10
                           ) -> Generator[ResponsesAgentStreamEvent, None, None]:
        for _ in range(max_iter):
            last_msg = messages[-1]
            if last_msg.get("role") == "assistant":
                return
            elif last_msg.get("type") == "function_call":
                yield self.handle_tool_call(last_msg, messages)
            else:
                yield from self.output_to_responses_items_stream(
                    chunks=self.call_llm(messages), aggregator=messages
                )

        yield ResponsesAgentStreamEvent(
            type="response.output_item.done",
            item=self.create_text_output_item("Max iterations reached. Stopping.", str(uuid4())),
        )

    def predict(self, request: ResponsesAgentRequest) -> ResponsesAgentResponse:
        outputs = [
            event.item
            for event in self.predict_stream(request)
            if event.type == "response.output_item.done"
        ]
        return ResponsesAgentResponse(output=outputs, custom_outputs=request.custom_inputs)

    def predict_stream(self, request: ResponsesAgentRequest
                       ) -> Generator[ResponsesAgentStreamEvent, None, None]:
        messages = self.prep_msgs_for_cc_llm([i.model_dump() for i in request.input])
        if SYSTEM_PROMPT:
            messages.insert(0, {"role": "system", "content": SYSTEM_PROMPT})
        yield from self.call_and_run_tools(messages=messages)

# Expose an MLflow model object
mlflow.openai.autolog()
AGENT = ToolCallingAgent(llm_endpoint=LLM_ENDPOINT_NAME, tools=TOOL_INFOS)
mlflow.models.set_model(AGENT)


In [0]:
# Import the freshly written module and do a quick smoke test
from agent import AGENT, UC_TOOL_NAMES, LLM_ENDPOINT_NAME

print("Tools wired:", UC_TOOL_NAMES)
resp = AGENT.predict({"input": [{"role": "user", "content": "What is 3*4 in Python?"}]})
print("Smoke test output items:", [o.model_dump() for o in resp.output][:1])

In [0]:
# Prepare resources and log the model as a run artifact (runs:/<run_id>/agent)
from mlflow.models.resources import DatabricksFunction, DatabricksServingEndpoint
from pkg_resources import get_distribution

resources = [DatabricksServingEndpoint(endpoint_name=LLM_ENDPOINT_NAME)]
for fn in UC_TOOL_NAMES:
    resources.append(DatabricksFunction(function_name=fn))

# Replace the current input_example with this:
input_example = {"input": [{"role": "user", "content": "How many days of period a woman experience around the age"}]}


with mlflow.start_run() as run:
    logged_info = mlflow.pyfunc.log_model(
        artifact_path="agent",
        python_model="agent.py",
        input_example=input_example,
        pip_requirements=[
            "databricks-openai",
            "backoff",
            f"databricks-connect=={get_distribution('databricks-connect').version}",
        ],
        resources=resources,
    )
    runs_source = f"runs:/{run.info.run_id}/agent"

print("Logged run URI:", runs_source)

In [0]:
loaded = mlflow.pyfunc.load_model(runs_source)
out = loaded.predict({"input": [{"role": "user", "content": "Predict next period start day for user 87"}]})


In [0]:
loaded.predict({"input": [{"role": "user", "content": "How many users have long bleed"}]})

In [0]:
loaded.predict({"input": [{"role": "user", "content": "Return overall statistics — total users analyzed, number and percentage of users with irregular patterns.Provide a short explanation of what “irregular cycle” means in data-tracking terms (educational only, not medical advice)"}]})

In [0]:
loaded.predict({"input": [{"role": "user", "content": 
"""
User 40 would like personalized guidance for the next 5 days.
Retrieve and analyze her last 40 cycle and symptom logs, along with her current cycle phase, life stage, and recorded preferences.
Identify key patterns in her recent data — including mood trends, symptom frequency, and energy levels — that may influence nutrition and wellness needs.
Based on her current phase, recommend 5 meal and wellness options, one for each of the next 5 days, that align with her physiological patterns and lifestyle.
Each recommendation should include both meal guidance and at least one complementary wellness activity (e.g., movement, rest, mindfulness, hydration).
Provide an explanation of the reasoning behind these recommendations, referencing how her phase, logs, and preferences informed each suggestion.
Use supportive, educational language (not medical advice).
"""}]})

In [0]:

loaded.predict({"input": [{"role": "user", "content": 
"""
User 18 will be out of town in Oct 6-10 to attend conferences.
Using her historical cycle data, current cycle phase, symptom and mood logs, and life stage, determine which cycle phase each of those travel days falls into.
Assess whether these days are optimal for travel given her typical patterns; if not ideal, explain risks and mitigations.
Predict her next period start date and state clearly if it is expected to occur during the 4-day trip.
Provide practical travel tips tailored to the predicted phases (energy, sleep, hydration, nutrition, comfort).
Recommend a day-by-day meal and activity plan for the 4 days (workload pacing, movement, meals, self-care), aligned to her phase and recent trends.
Include your reasoning and reference any relevant trends you used (e.g., average cycle length, typical symptom timing). Use supportive, non-medical language.
"""}]})

In [0]:
loaded.predict({"input": [{"role": "user", "content": 
"""
User 30 is planning a 5-day vacation during the first two weeks of December.
Use her historical cycle data and symptom logs to determine what phase she is likely to be in around that time.
Based on her typical patterns and recent wellness indicators, assess whether this period is optimal for travel.
If not, recommend the best alternative windows for travel within December.
Provide practical guidance on how she can manage her wellness, energy, and mood during the trip if she travels anyway.
Explain your reasoning and reference relevant cycle trends or data patterns.
"""}]})

In [0]:
# Register to Unity Catalog by creating a new version from runs_source
from mlflow.tracking import MlflowClient

client = MlflowClient()
try:
    client.get_registered_model(UC_MODEL_NAME)
    print("UC registered model exists:", UC_MODEL_NAME)
except Exception:
    client.create_registered_model(UC_MODEL_NAME)
    print("Created UC registered model:", UC_MODEL_NAME)

mv = client.create_model_version(name=UC_MODEL_NAME, source=runs_source)
print(f"✅ Created UC model version {mv.version} for {UC_MODEL_NAME}")

In [0]:
# (Optional) Deploy to a serving endpoint if you want
try:
    from databricks import agents
    agents.deploy(UC_MODEL_NAME, mv.version, scale_to_zero=True, tags={"endpointSource": "playground"})
    print("Deployment requested for:", UC_MODEL_NAME, f"v{mv.version}")
except Exception as e:
    print("Skipping deploy (agents SDK not available or insufficient permissions):", e)

In [0]:
# Invoke your Databricks serving endpoint
import requests
from databricks.sdk import WorkspaceClient

UC_MODEL_NAME = "wwc2025.period_pipeline.wellness_agent"

# If you already know it, hardcode:
ENDPOINT_NAME = "agents_wwc2025-period_pipeline-wellness_agent"   # e.g. "agents-wellness-agent"

w = WorkspaceClient()
tmp_token = w.tokens.create(lifetime_seconds=2400).token_value

url = f"{w.config.host}/serving-endpoints/{ENDPOINT_NAME}/invocations"
headers = {"Authorization": f"Bearer {tmp_token}"}

payload = {
  "input": [{"role": "user",
             "content": "For user 40, analyze last 40 logs and current cycle phase, then recommend 5 luteal-phase meal & wellness options."}],
  "temperature": 0.2
}

resp = requests.post(url, headers=headers, json=payload, timeout=120)
resp.raise_for_status()
print(resp.json())
